In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
from google.colab import files
final =files.upload()

In [ ]:
data =[]
sentences = ""
with open('final.txt','r',encoding = "utf-8",errors  ="ignore") as corpus:
    for line in corpus:
        sentences = sentences + line.lower()


In [ ]:
#!"#$%&'()*+, -./:;<=>?@[\]^_`{|}~
punc = ['#','"','$','%','&','(',')','*','+','-','/','<','=','>','@','[',']','\\','^','_','`','{','}','|','~','\n',',',"'"]
for p in punc:
    sentences = sentences.replace(p," ")
    
sentences = sentences.replace("?",".")
sentences = sentences.replace("!",".")
sentences = sentences.replace(":",".")
sentences = sentences.replace(";",".")

data = sentences.split(".")
data = [d for d in data if len(d.split()) >= 10]

In [ ]:

print(len(data))

In [ ]:
vocab_size = 10000
embedding_dim = 32

tokenizer = Tokenizer(num_words = vocab_size,oov_token ="<oov>")
tokenizer.fit_on_texts(data)

In [ ]:
print(tokenizer.word_index)

In [ ]:
word_index = tokenizer.word_index
word_index = {word:index for word,index in word_index.items() if index<=vocab_size}
#rint(word_index["lapland"])
index_to_word = { index:word for word,index in word_index.items()}
print(len(word_index))

In [ ]:
refined_data = []
for s in data:
    tokens = s.split()
    sen = ""
    for word in tokens:
        if word in word_index:
            sen = sen+" "+ word
        else:
            sen = sen+" "+"<oov>"
    refined_data.append(sen)

print(refined_data[10:40])    

In [ ]:
def generate_data(refined_data,word_index,window_size = 3):
    X =[]
    Y =[]
    for s in refined_data:
        tokens = [word_index[word] for word in s.split()]
        for i,context in enumerate(tokens):
            for j in range(max(i-window_size,0),i):
                X.append(tokens[i])
                Y.append(tokens[j])
            for j in range(i+1,min(i+window_size,len(tokens))):
                X.append(tokens[i])
                Y.append(tokens[j])
    return X,Y
                
            

In [ ]:
X,Y = generate_data(refined_data,word_index,window_size = 3)

In [ ]:
print(X[:30])
print(Y[:30])

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1,embedding_dim,input_length = 1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(embedding_dim,activation = 'relu'),
    tf.keras.layers.Dense(vocab_size+1,activation = 'softmax')
])

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
model.summary()

In [ ]:
X_train = X[:8000000]
Y_train = Y[:8000000]
X_val = X[8000000:]
Y_val = Y[8000000:]
model.fit(X_train,Y_train,epochs=10,batch_size = 10000,validation_data  = (X_val,Y_val))

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_sentence(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [ ]:
import io

out_v = io.open('vecs_10000.tsv', 'w', encoding='utf-8')
out_m = io.open('meta_10000.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [ ]:
files.download('vecs_10000.tsv')
files.download('meta_10000.tsv')

In [ ]:
print(len(X))